In [3]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import geopandas as gpd
import time
from math import sqrt
import pandas as pd
pd.set_option('precision', 10)
%matplotlib inline

from shapely.geometry import Point, LineString, Polygon, MultiPoint
from shapely.ops import split, nearest_points

import street_network_functions as snf
import utilities as uf
from importlib import reload
import seaborn as sns
pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from shapely.ops import linemerge, unary_union, polygonize

C:\Users\g_filo01\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysalnext` package. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [1]:
city_name = 'London'
IoC_output ='C:/Users/g_filo01/sciebo/scripts/Image of the City/Outputs/'+city_name+'/large_network/'+city_name
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

In [4]:
nodes = gpd.read_file(IoC_output+'_nodes.shp')
edges = gpd.read_file(IoC_output+'_edges.shp')

In [5]:
nd = gpd.read_file("C:/Users/g_filo01/sciebo/Tools/Mason/sim/app/geo/LondonDistricts/data/London_nodes.shp")

In [9]:
def dist_to_gdf(line, gpd):
    gpd = gpd.copy()
    gpd['Dist'] = gpd.apply(lambda row: line.distance(row.geometry),axis=1)
    geoseries = gpd.loc[gpd['Dist'].argmin()]
    index = geoseries.name
    return index

In [13]:
index_geo = edges.columns.get_loc("geometry")+1
edges['district'] = 0
spatial_index = d.sindex # spatial index

for row in edges.itertuples():
    uf.print_row(row.Index)
    line = row[index_geo]
    n = line.buffer(70)
    possible_matches_index = list(spatial_index.intersection(n.bounds))
    pm = d.iloc[possible_matches_index].copy()
    if (len(pm)) == 0: continue
    index = dist_to_gdf(line, pm)
    edges.at[row.Index, 'district'] = d.loc[index].district

at row: 27812

In [ ]:
nodes['district'] = None
index_geo = nodes.columns.get_loc("geometry")+1  
spatial_index = edges.sindex # spatial index

for row in nodes.itertuples():
    point = row[index_geo]
    n = point.buffer(20)
    possible_matches_index = list(spatial_index.intersection(n.bounds))
    pm = edges.iloc[possible_matches_index].copy()
    dist = uf.dist_to_gdf(point, pm)
    
    district = edges.loc[dist[1]]['district'] 
    nodes.at[row[0], 'district'] = district

In [14]:
edges.to_file(IoC_output+'_edges_heck.shp', driver='ESRI Shapefile')

# Gateways Identification

In [4]:
edges.set_index('streetID', drop = False, inplace = True, append = False)
del edges.index.name

In [6]:
# per each node which is connected to other nodes in other region (gateways)
# fill a field whit a list of accessible districts

index_u = edges.columns.get_loc("u")+1 
index_v = edges.columns.get_loc("v")+1 
nodes['regionsTo'] = None
nodes.set_index('nodeID', drop = False, inplace = True, append = False)
del nodes.index.name

for row in edges.itertuples():
    node_ud = nodes.loc[row[index_u]]['district']
    node_vd = nodes.loc[row[index_v]]['district']
    
    if node_ud == node_vd: continue
    else:
        gw_u = nodes.loc[row[index_u]]['regionsTo']
        if type(gw_u) == list: 
            if node_vd in gw_u: pass
            else: gw_u.append(node_vd)
        elif gw_u == None: gw_u = [node_vd]
        nodes.set_value(row[index_u], 'regionsTo', gw_u)
        
        gw_v = nodes.loc[row[index_v]]['regionsTo']
        if type(gw_v) == list: 
            if node_ud in gw_v: pass
            else: gw_v.append(node_ud)
        elif gw_v == None: gw_v = [node_ud]

        nodes.set_value(row[index_v], 'regionsTo', gw_v)

Wall time: 10.5 s


In [12]:
nodes.head()

,nodeID,x,y,height,weight,Bc,Rc400,Rc600,Bc400,Bc600,district,CM,Bc_multi,district_O,geometry,regionsTo
0,0,528190.105,182301.253,2,1.000,781987.000,229.000,1963.000,6594.000,899.000,49,1,773661.000,49,POINT (528190.1045659232 182301.2530380917),None
1,1,528181.393,182309.512,2,2.000,766179.000,227.000,1580.000,5702.000,974.000,49,0,757743.000,49,POINT (528181.3931032321 182309.5123659582),None
2,2,528242.880,182222.044,2,1.000,166516.000,344.000,2845.000,1635.000,168.000,49,1,171230.000,49,POINT (528242.879545405 182222.0442442839),[39]
3,3,527934.138,182291.062,2,2.000,78532.000,394.000,1876.000,3235.000,325.000,49,1,75367.000,49,POINT (527934.1384294253 182291.061583835),None
4,4,527911.906,182259.601,2,1.000,254088.000,498.000,2021.000,3009.000,406.000,49,0,253773.000,49,POINT (527911.9064132097 182259.6007812982),None


In [11]:
nodes_string = nodes.copy()
nodes_string['regionsTo'] = nodes_string['regionsTo'].astype(str)
nodes_string.to_file(IoC_output+'_nodes.shp', driver='ESRI Shapefile')

# Polygonising Edges for Districts visualisation

In [ ]:
unionStreet = streets.geometry.unary_union
envelope = unionStreet.envelope

districts = list(edges.district.unique())
polygons = []
data = []

for i in districts:
    tmp = edges[edges.district == i]
    u = tmp.geometry.unary_union
    p = polygonize_full(u)
    final = p[0].buffer(50).simplify(0.95, preserve_topology = False)
    if final.geom_type == 'MultiPolygon': 
        area = 0
        for p in final:
            if p.area > area: 
                area = p.area
                best = p
        final = best
    polygons.append(final)
    data.append(i)
    
districts_polygonized = gpd.GeoDataFrame(data, crs=crs, geometry = polygons)
districts_polygonized.rename(columns={0:'district'}, inplace = True)
districts_polygonized.to_file(IoC_output+'_districts_polygonized.shp', driver='ESRI Shapefile')

In [194]:
## Main functions

def gateways_exit(current_location, destination, current_region, visited_regions, gateways_gdf, nodes_gdf, edges_gdf):
    
    nodes_gdf = nodes_gdf.copy()
    geoCL = nodes_gdf.loc[current_location].geometry
    geoDE = nodes_gdf.loc[destination].geometry
    
    index_regionsTo = nodes_gdf.columns.get_loc("regionsTo")+1 
    
    # gateways in 'current_region' and in CM
    gateways = gateways_gdf[(gateways_gdf.district_m.isin(current_region)) & (gateways_gdf.CM == 1)].copy()
    gateways.drop(current_location, axis = 1, inplace = True, errors = 'ignore')
    if len(gateways) == 0: return gateways
    
    # distance rule
    distance = geoCL.distance(geoDE)
    gateways['dist'] = gateways.apply(lambda row: geoDE.distance(row.geometry),axis=1)
    gateways = gateways[gateways.dist < distance]
    if len(gateways) == 0: return gateways
    
    # gateway direction rule
    gateways['direction'] = gateways.apply(lambda row: compare_angle(geoCL, geoDE, row.geometry),axis=1)
    gateways = gateways[gateways['direction'] == True]
    if len(gateways) == 0: return gateways
    
    # check if there's access to good regions from these gateways
    possible_regions = [item for sublist in gateways.regionsTo for item in sublist]
    possible_regions = list(set(possible_regions))
    good_candidates = [item for item in possible_regions if item not in visited_regions]
    if len(good_candidates) == 0: return gateways.iloc[0:0] 
    
    # keep only good regions
    for g in gateways.itertuples():
        p_regions = g[index_regionsTo] # possible regions
        p_regions = [i for i in p_regions if i in good_candidates]
        if len(p_regions) == 0: gateways.drop(g[0], axis = 0, inplace = True)
        else: gateways.set_value(g[0], 'regionsTo', p_regions)
    
    return gateways



def gateways_entry(current_location, exit, destination, gateways_gdf, nodes_gdf, edges_gdf, regions):
    
    geoCL = nodes_gdf.loc[current_location].geometry
    geoDE = nodes_gdf.loc[destination].geometry
    
    # find entries from 'exit'
    gdf = edges_gdf[(edges_gdf.u == exit) | (edges_gdf.v == exit)].copy()
    entries = list(gdf['u'].unique())+list(gdf['v'].unique())
    entries = list(set(entries))
    entries.remove(exit)
    if current_location in entries: entries.remove(current_location)
        
    gateways = gateways_gdf[(gateways_gdf.nodeID.isin(entries)) & (gateways_gdf.district_m.isin(regions))].copy()
    if len(gateways) == 0: return gateways
    
    # gateway direction rule
    gateways['direction'] = gateways.apply(lambda row: compare_angle(geoCL, geoDE, row.geometry),axis=1)
    gateways = gateways[gateways['direction'] == True]

    return(gateways)
    
    
def compare_angle(origin, destination, node):

    distance = origin.distance(destination)
    originCoord = list(origin.coords[0])
    destinationCoord = list(destination.coords[0])
    nodeCoord = list(node.coords[0]) 
        
    zeroCoord = uf.get_coord_angle(originCoord, distance = distance, angle = 0)
    line0 = ((originCoord), (zeroCoord))
    lineDest = ((originCoord), (destinationCoord))
    lineNode = ((originCoord), (nodeCoord))

    # east # west
    if destinationCoord[0] > originCoord[0]: angleDest = uf.ang(line0, lineDest)
    else: angleDest = 180+(180-uf.ang(line0, lineDest))
    ang_coordDestA = angleDest-71
    ang_coordDestB = angleDest+71
    
    if ang_coordDestA < 0: ang_coordDestA = 360 + ang_coordDestA
    if ang_coordDestB > 360: ang_coordDestB = ang_coordDestB -360
    
    if nodeCoord[0] > originCoord[0]: angleNode = uf.ang(line0, lineNode)
    else: angleNode = 180+(180-uf.ang(line0, lineNode))
    if ((angleNode < ang_coordDestB) & (ang_coordDestA> 180) & (angleNode > 0) & (angleNode < 180) 
        & (angleNode < ang_coordDestB)): return True
    elif ((angleNode > ang_coordDestB) & (angleNode > ang_coordDestA) & 
        (ang_coordDestB < 180) & (angleNode > 180)): return True
    elif (angleNode < ang_coordDestB) & (angleNode > ang_coordDestA): return True
    else: return False
        
    

In [660]:
def graph_from_region(region, nodes_gdf, edges_gdf):
    nodes_region = nodes[nodes.district_m == region].copy()
    edges_region = edges[edges.u.isin(list(nodes_region.nodeID)) | edges.u.isin(list(nodes_region.nodeID))].copy()
    keep =  list(edges_region['u'].unique())+list(edges_region['v'].unique())
    keep = list(set(keep))
    nodes_region = nodes_gdf[nodes.nodeID.isin(keep)]
    Dg = snf.get_dual_graph(nodes_region, edges_region, 'rad')